## Install additional modules
This is necessary only because we are using a docker image from jupter called 'docker-stacks' that doesn't including pymongo, so we just have to do this.

In [6]:
!pip install pymongo
!pip install mongoengine

    100% |################################| 348kB 194kB/s 
  Running setup.py bdist_wheel for mongoengine
  Stored in directory: /home/jovyan/.cache/pip/wheels/ec/a9/25/18ff0820fc8e20e6d6f88ae3c6bada6194d9a48ace852bb409
Successfully built mongoengine


## Load Data
The step step here is to generate some sample data.  Feel free to adjust this as needed.

In [12]:
# Import the required module
from pymongo import MongoClient

c = MongoClient("mongo")
db = c.main_study
response = db.response

In [13]:
# Make up some data

from datetime import datetime, timedelta
import random

# fake
users = ['alfazo', 'bieberhold', 'little_pope', 'buttons', 'lpasteur', 'kevin', 'tommyboy', 'bstreisand', 'captain', 'twain']

for counter, idx in enumerate(range(10)):
    
    record = {
            "timestamp": datetime.today() - timedelta(idx),
            "device_id": random.randint(1000000, 9999999),
            "username": random.choice(users),
            "geo": {
                "lat": 13.4080,
                "lon": 52.5186
              }
        }
    
    response.insert_one(record)


In [14]:
# preview what we have here

for record in response.find().limit(5):
    print(record)

{'geo': {'lon': 52.5186, 'lat': 13.408}, 'timestamp': datetime.datetime(2015, 12, 16, 19, 22, 12, 28000), 'device_id': 3475006, 'username': 'buttons', '_id': ObjectId('5671b9e4c07f43001312ec68')}
{'geo': {'lon': 52.5186, 'lat': 13.408}, 'timestamp': datetime.datetime(2015, 12, 15, 19, 22, 12, 35000), 'device_id': 7299786, 'username': 'little_pope', '_id': ObjectId('5671b9e4c07f43001312ec69')}
{'geo': {'lon': 52.5186, 'lat': 13.408}, 'timestamp': datetime.datetime(2015, 12, 14, 19, 22, 12, 35000), 'device_id': 9748881, 'username': 'alfazo', '_id': ObjectId('5671b9e4c07f43001312ec6a')}
{'geo': {'lon': 52.5186, 'lat': 13.408}, 'timestamp': datetime.datetime(2015, 12, 13, 19, 22, 12, 36000), 'device_id': 2938920, 'username': 'little_pope', '_id': ObjectId('5671b9e4c07f43001312ec6b')}
{'geo': {'lon': 52.5186, 'lat': 13.408}, 'timestamp': datetime.datetime(2015, 12, 12, 19, 22, 12, 36000), 'device_id': 5613267, 'username': 'bstreisand', '_id': ObjectId('5671b9e4c07f43001312ec6c')}


# Mongo Engine

In [25]:
from mongoengine import connect, Document, StringField, DateTimeField

In [11]:
connect('main_study', host='mongo')

MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

### Define the model

In [28]:
class Response(Document):
    username = StringField()
    geo = StringField()
    timestamp = DateTimeField()
    device_id = StringField()

### Query the database

In [29]:
for response in Response.objects:
    print(response.username)

buttons
little_pope
alfazo
little_pope
bstreisand
captain
bieberhold
buttons
little_pope
buttons
